# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

ModuleNotFoundError: No module named 'pymongo'

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
print(mongo.list_database_names())

ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 675f93062a8533f2ee5867ac, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 61] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']
print(db.list_collection_names())

In [ ]:
# review the collections in our database
collections = db.list_collection_names()
for collection in collections:
    print(collection)

In [ ]:
# assign the collection to a variable
establishments = db['establishments']
document = establishments.find_one()
pprint(document)

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {"hygiene_score": 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments with a hygiene score of 20: {count}")

# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint(first_document)

In [ ]:
# Convert the result to a Pandas DataFrame
results = list(establishments.find(query))
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print(df.head(10))

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {
    "LocalAuthority": "London",
    "RatingValue": {"$gte": 4}
}
# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments in London with a RatingValue >= 4: {count}")

# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint(first_document)

In [ ]:
# Convert the result to a Pandas DataFrame
results = list(establishments.find(query))
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print(df.head(10))

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.5074  # Example latitude for London
longitude = -0.1278  # Example longitude for London

query = {
    "RatingValue": 5,
    "location": {
        "$geoWithin": {
            "$centerSphere": [[longitude, latitude], degree_search / 3963.0]  # Radius in miles
        }
    }
}
sort = [("hygiene_score", 1)]
limit = 10  # Adjust the limit as needed

# Print the results
results = establishments.find(query).sort(sort).limit(limit)
for result in results:
    pprint(result)

In [ ]:
# Convert result to Pandas DataFrame
results = establishments.find(query).sort(sort).limit(limit)
df = pd.DataFrame(list(results))
print(f"Number of rows in DataFrame: {len(df)}")
print(df.head(10))

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
matchQ = {'$match':{'scores.Hygiene':0}}
# 2. Groups the matches by Local Authority
groupQ = {'$group':{'_id':'$LocalAuthorityName','count':{'$sum':1}}}
# 3. Sorts the matches from highest to lowest
sortQ = {'$sort':{'count':-1}}
pipeline = [matchQ,groupQ,sortQ]
results = list(establishments.aggregate(pipeline))
# Print the number of documents in the result
print("Number of documents: ", len(results))
# Print the first 10 results
pprint(results[0:10])

In [ ]:
# Convert the result to a Pandas DataFrame
hy0CountDf = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print(f'Number of rows: {len(hy0CountDf)}')
# Display the first 10 rows of the DataFrame
hy0CountDf.head(10)